In [2]:
import numpy as np
import pandas as pd
import os 

from matplotlib import pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

### LOAD DATASETS & PLANNING

In [35]:
raw_datasets_path = os.path.abspath(os.path.join('raw_datasets'))
cleaned_datasets_path = os.path.abspath(os.path.join('cleaned_datasets'))

In [36]:
os.path.join(raw_datasets_path, 'jabodetabek_house_price.csv')

'c:\\Users\\balse\\Documents\\VScode\\STIMA-Makalah\\STIMA-Makalah-Analysis\\raw_datasets\\jabodetabek_house_price.csv'

In [37]:
# LOAD 2023 PROPERTY SALE DATA
def load_data(filename):
    return pd.read_csv(os.path.join(raw_datasets_path, filename))

df_sale_2023_raw = load_data('jabodetabek_house_price.csv')
df_sale_raw = load_data('rumah123_sale_properties.csv')
df_rent_raw = load_data('rumah123_rent_properties.csv')

In [38]:
# CLEANING PROCESS:
'''
- remove building_age feature
- re-order columns
- rename + re-order sale_2023 data
- check the data types of each column
    - if not numeric, convert to numeric (e.g. bathrooms, bedrooms)
- convert price_in_rp, electricity, land_size, building_size  to numeric

- check city unique values. Only keep Jabodetabek cities (Jakarta, Bogor, Depok, Tangerang, Bekasi). -> clean (e.g. Bandung writen as Badung)

- remove whitespaces for categorical features. e.g. replace spaces with underscores " " -> "_".
(just for visualization purposes, XGBoost does not care about this)


- identify duplicated columns
- Handling null/na values: replace/remove

- feature understanding (univariate analysis): histogram, KDE, boxplot (identifying outliers).
    checking and removing outliers.

- Further cleaning based on special cases found in the exploratory data analysis (EDA).
'''

# DATA PREPARATION (XGBoost)
'''
- data normalization (e.g. price and building area scale are far apart)
- One Hot Encoding (categorical -> numerical for XGBoost)
'''

'\n- data normalization (e.g. price and building area scale are far apart)\n- One Hot Encoding (categorical -> numerical for XGBoost)\n'

### DATA CLEANING

In [39]:
df_rent_raw.shape

(5671, 17)

In [40]:
# Remove building_age feature
df_sale = df_sale_raw.drop(columns=['building_age'], errors='ignore')
df_rent = df_rent_raw.drop(columns=['building_age'], errors='ignore')

# re-order columns
df_sale = df_sale[['url', 'title', 'price_in_rp', 'district', 'city' , 'bedrooms', 'bathrooms', 'land_size_m2', 'building_size_m2','certificate', 'electricity_power', 'property_condition', 'floor_count', 'description']]

df_rent = df_rent[['url', 'title', 'price_in_rp', 'district', 'city' , 'bedrooms', 'bathrooms', 'land_size_m2', 'building_size_m2','certificate', 'electricity_power', 'property_condition', 'floor_count', 'description']]
df_rent.rename(columns={'price_in_rp': 'rent_price_per_year'}, inplace=True)

In [41]:
# rename + re-order sale_2023 data
df_sale_2023 = df_sale_2023_raw.rename(columns={
    'url': 'url', 
    'price_in_rp': 'price_in_rp', 
    'title': 'title', 
    # 'address': 'location', 
    'district': 'district', 
    'city': 'city', 
    # 'lat',
    # 'long', 
    'facilities': 'description', 
    # 'property_type', 
    # 'ads_id', 
    'bedrooms': 'bedrooms',
    'bathrooms': 'bathrooms', 
    'land_size_m2': 'land_size_m2', 
    'building_size_m2': 'building_size_m2', 
    # 'carports'
    'certificate': 'certificate', 
    'electricity': 'electricity_power', 
    # 'maid_bedrooms', 
    # 'maid_bathrooms',
    'floors': 'floor_count', 
    # 'building_age', 
    # 'year_built', 
    'property_condition': 'property_condition',
    # 'building_orientation', 
    # 'garages', 
    # 'furnishing'    
}).copy()

df_sale_2023 = df_sale_2023[['url', 'title', 'price_in_rp', 'district', 'city' , 'bedrooms', 'bathrooms', 'land_size_m2', 'building_size_m2', 'certificate', 'electricity_power', 'property_condition', 'floor_count', 'description']]

In [42]:
# check the data types of each column prior to conversion
print(df_sale.dtypes)
print(df_rent.dtypes)

url                    object
title                  object
price_in_rp            object
district               object
city                   object
bedrooms              float64
bathrooms             float64
land_size_m2           object
building_size_m2       object
certificate            object
electricity_power      object
property_condition     object
floor_count           float64
description            object
dtype: object
url                     object
title                   object
rent_price_per_year     object
district                object
city                    object
bedrooms               float64
bathrooms              float64
land_size_m2            object
building_size_m2        object
certificate             object
electricity_power       object
property_condition      object
floor_count            float64
description             object
dtype: object


In [43]:
# NUMERIC DATA TYPE CONVERSION (2025 DATA)

# --- price_in_rp ---
for i, val in enumerate(df_sale['price_in_rp']):
    if isinstance(val, str):
        val = val.replace('Rp', '').replace(',', '.')
        if 'Miliar' in val:
            val = val.replace('Miliar', '').strip()
            val = float(val) * 1_000_000_000
        elif 'Juta' in val:
            val = val.replace('Juta', '').strip()
            val = float(val) * 1_000_000
        df_sale.at[i, 'price_in_rp'] = val
    # print(i, val)

for i, val in enumerate(df_rent['rent_price_per_year']):
    if isinstance(val, str):
        val = val.replace('Rp', '').replace(',', '.')
        if '/bulan' in val or 'per bulan' in val:
            val = val.replace('/bulan', '').replace('per bulan', '').strip()
            period = 12
        elif '/tahun' in val or 'per tahun' in val:
            val = val.replace('/tahun', '').replace('per tahun', '').strip()
            period = 1
        elif '/hari' in val or 'per hari' in val:
            val = val.replace('/hari', '').replace('per hari', '').strip()
            period = 365
            
        if 'Miliar' in val:
            val = val.replace('Miliar', '').strip()
            val = float(val) * 1_000_000_000 * period
        elif 'Juta' in val:
            val = val.replace('Juta', '').strip()
            val = float(val) * 1_000_000 * period
        elif 'Ribu' in val:
            val = val.replace('Ribu', '').strip()
            val = float(val) * 1_000 * period
        df_rent.at[i, 'rent_price_per_year'] = float(val)
    # print(i, val)


# --- land_size_m2 ---
if df_sale['land_size_m2'].dtype == 'object':
    # Remove 'm²' and convert to float
    df_sale['land_size_m2'] = df_sale['land_size_m2'].str.replace('m²', '').str.replace(',', '.').astype(float)
if df_rent['land_size_m2'].dtype == 'object':
    # Remove 'm²' and convert to float
    df_rent['land_size_m2'] = df_rent['land_size_m2'].str.replace('m²', '').str.replace(',', '.').astype(float)

# --- building_size_m2 ---
if df_sale['building_size_m2'].dtype == 'object':
    # Remove 'm²' and convert to float
    df_sale['building_size_m2'] = df_sale['building_size_m2'].str.replace('m²', '').str.replace(',', '.').astype(float)
if df_rent['building_size_m2'].dtype == 'object':
    # Remove 'm²' and convert to float
    df_rent['building_size_m2'] = df_rent['building_size_m2'].str.replace('m²', '').str.replace(',', '.').astype(float)

# --- electricity_power ---
if df_sale['electricity_power'].dtype == 'object':
    # Remove 'Watt' and convert to float
    df_sale['electricity_power'] = df_sale['electricity_power'].str.replace('Lainnya', '0').str.replace('Watt', '').str.replace(',', '.').astype(float)
if df_rent['electricity_power'].dtype == 'object':
    # Remove 'Watt' and convert to float
    df_rent['electricity_power'] = df_rent['electricity_power'].str.replace('Lainnya', '0').str.replace('Watt', '').str.replace(',', '.').astype(float)


# CATEGORICAL DATA CLEANING
# standardize certificate values
df_sale['certificate'] = df_sale['certificate'].replace(
    [
        'PPJB', 'Hak Sewa'
    ], 
    'Lainnya')
df_rent['certificate'] = df_rent['certificate'].replace(
    [
        'PPJB', 'Hak Sewa'
    ],
    'Lainnya')

In [44]:
# check the data types of each column after conversion
print(df_sale.dtypes, end='\n\n')
print(df_rent.dtypes)

url                    object
title                  object
price_in_rp            object
district               object
city                   object
bedrooms              float64
bathrooms             float64
land_size_m2          float64
building_size_m2      float64
certificate            object
electricity_power     float64
property_condition     object
floor_count           float64
description            object
dtype: object

url                     object
title                   object
rent_price_per_year     object
district                object
city                    object
bedrooms               float64
bathrooms              float64
land_size_m2           float64
building_size_m2       float64
certificate             object
electricity_power      float64
property_condition      object
floor_count            float64
description             object
dtype: object


In [45]:
# NUMERIC DATA TYPE CONVERSION (2023 HOUSE SALES DATA)

# --- electricity_power ---
'''
- convert mah to watt. 1 Watt = 1 mAh (this is an assumption because somehow the range of electricity between watt in 2025 data and mah in 2023 data is similar)
'''
if df_sale_2023['electricity_power'].dtype == 'object':
    # Replace 'lainnya' with 0, remove 'mah', replace ',' with '.' and convert to float
    df_sale_2023['electricity_power'] = df_sale_2023['electricity_power'].str.replace('lainnya', '0').str.replace('mah', '').str.replace(',', '.').astype(float)

# --- property condition mapping ---
property_condition_mapping = {
    'baru': 'Baru',
    'bagus sekali': 'Baru',
    'bagus': 'Bagus',
    'sudah renovasi': 'Sudah Renovasi',
    'semi furnished': 0,
    'butuh renovasi': 'Butuh Renovasi',
    'unfurnished': 0
}
if df_sale_2023['property_condition'].dtype == 'object':
    df_sale_2023['property_condition'] = df_sale_2023['property_condition'].map(property_condition_mapping)

# CATEGORICAL DATA CLEANING
# standardize certificate values
df_sale_2023['certificate'] = df_sale_2023['certificate'].replace(
    [
        'shm - sertifikat hak milik', 'hgb - hak guna bangunan', 'lainnya (ppjb,girik,adat,dll)', 'hp - hak pakai'
    ], 
    [
        'SHM', 'HGB', 'Lainnya', 'Hak Pakai'
    ])

# trim spaces and whitespaces for categorical features
df_sale_2023['district'] = df_sale['district'].str.strip()
df_sale_2023['city'] = df_sale_2023['city'].str.strip()

In [46]:
# check city unique values. Only keep Jabodetabek cities (Jakarta, Bogor, Depok, Tangerang, Bekasi).
df_sale.query("city == ['Bekasi', 'Bogor', 'Depok', 'Jakarta Barat', 'Jakarta Selatan', 'Jakarta Utara', 'Jakarta Timur', 'Jakarta Pusat', 'Tangerang']", inplace=True)
df_rent.query("city == ['Bekasi', 'Bogor', 'Depok', 'Jakarta Barat', 'Jakarta Selatan', 'Jakarta Utara', 'Jakarta Timur', 'Jakarta Pusat', 'Tangerang']", inplace=True)

In [47]:
# DUPLICATE HANDLING
'''
there's no duplicates in the dataset: df_sale, df_rent, df_sale_2023
'''

# NULL VALUES HANDLING
    # if we don't have time, just dropna. But if we do, do this:
'''
df_sale:
- check ourself from the image or description: bedrooms, bathrooms, floor_count, property_condition

df_rent:
- check ourself from the image or description: bedrooms, bathrooms, land_size_m2, building_size_m2
- fill null: floor_count = 1 (check first)
- drop null: property_condition

df_sale_2023:
- drop null: location, land_size_m2, building_size_m2, certificate, property_condition, floor_count
'''

# df_sale:
df_sale.dropna(subset=df_sale.columns[df_sale.isnull().any()].tolist(), inplace=True)

# df_rent:
df_rent.dropna(subset=df_rent.columns[df_rent.isnull().any()].tolist(), inplace=True)

# df_sale_2023:
df_sale_2023.dropna(subset=df_sale_2023.columns[df_sale_2023.isnull().any()].tolist(), inplace=True)

In [48]:
# REMOVE PRICE OUTLIERS using IQR method
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# remove outliers for price_in_rp
df_sale = remove_outliers_iqr(df_sale, 'price_in_rp')
df_rent = remove_outliers_iqr(df_rent, 'rent_price_per_year')
df_sale_2023 = remove_outliers_iqr(df_sale_2023, 'price_in_rp')

In [49]:
# SAVE CLEANED DATA
os.makedirs(cleaned_datasets_path, exist_ok=True)

def save_cleaned_data(df, filename):
    df.to_csv(os.path.join(cleaned_datasets_path, filename), index=False)

save_cleaned_data(df_sale, 'rumah123_2025_sale_properties_cleaned.csv')
save_cleaned_data(df_rent, 'rumah123_2025_rent_properties_cleaned.csv')
save_cleaned_data(df_sale_2023, 'rumah123_2023_sale_properties_cleaned.csv')

In [50]:
# cleaned dataframes
df_sale, df_rent, df_sale_2023

(                                                    url  \
 1     https://www.rumah123.com/properti/bekasi/hos38...   
 2     https://www.rumah123.com/properti/bogor/hos393...   
 3     https://www.rumah123.com/properti/bogor/hos392...   
 4     https://www.rumah123.com/properti/depok/hos392...   
 5     https://www.rumah123.com/properti/depok/hos392...   
 ...                                                 ...   
 2589  https://www.rumah123.com/properti/tangerang/ho...   
 2590  https://www.rumah123.com/properti/tangerang/ho...   
 2592  https://www.rumah123.com/properti/jakarta-utar...   
 2599  https://www.rumah123.com/properti/jakarta-utar...   
 2600  https://www.rumah123.com/properti/jakarta-utar...   
 
                                                   title   price_in_rp  \
 1     Rumah Hadap Utara 13 menit ke Plaza Taman Hara...   590000000.0   
 2     Rumah LT 120 Hadap Selatan 15 menit ke Metropo...  2040000000.0   
 3     Rumah Minimalis Murah Akses Dekat Stasiun Bojo...

### DATA PREPARATION FOR MODELLING